In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.linalg import khatri_rao

#for debug
import matplotlib.pyplot as plt
import time as tm

In [2]:

def my_map(X):
    X = 1 - 2* np.array(X)
    X = np.cumprod(X[:, ::-1], axis=1)[:, ::-1].transpose()
    return np.concatenate((khatri_rao(X, X), X), axis = 0).transpose()


def my_fit(X_train, Y_train ):
    phi_X = my_map(X_train)
    model = LinearSVC(C = 3.4304692863149193, loss='squared_hinge', dual=False)
    model.fit(phi_X, np.array(Y_train))
    return model.coef_.transpose().flatten(), model.intercept_



In [3]:
my_map(np.array([[1.2]]))

array([[ 1.96, -1.4 ]])

In [4]:
Z_trn = np.loadtxt( "dummy/secret_train.dat" )
Z_tst = np.loadtxt( "dummy/secret_test.dat" )

n_trials = 5

d_size = 0
t_train = 0
t_map = 0
acc = 0

In [5]:
train_data = np.loadtxt("dummy\secret_train.dat", delimiter=" ")
trainX, trainY = train_data.transpose()[:-1].transpose(), train_data.transpose()[-1].transpose()

test_data = np.loadtxt("dummy\secret_test.dat", delimiter=" ")
testX, testY = test_data.transpose()[:-1].transpose(), test_data.transpose()[-1].transpose()

In [6]:
for t in range( n_trials ):
	tic = tm.perf_counter()
	w, b = my_fit( Z_trn[:, :-1], Z_trn[:,-1] )
	toc = tm.perf_counter()
	t_train += toc - tic
	d_size += w.shape[0]

	tic = tm.perf_counter()
	feat = my_map( Z_tst[:, :-1] )
	toc = tm.perf_counter()
	t_map += toc - tic

	scores = feat.dot( w ) + b
	pred = np.zeros_like( scores )
	pred[scores > 0] = 1
	acc += np.average( Z_tst[ :, -1 ] == pred )

In [7]:
pred.shape, Z_tst[ :, -1 ].shape

((10000,), (10000,))

In [8]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials

print( d_size, t_train, t_map, 1 - acc )

1056.0 6.679175140000001 0.07675907999999936 0.00660000000000005
